In [46]:
#! pip install datasets transformers[sentencepiece]

Si estás abriendo este cuaderno de forma local, asegúrate de tener instalada la última versión de Datasets y una instalación de origen de Transformers en tu entorno.

In [47]:
from transformers.utils import send_example_telemetry

send_example_telemetry("tokenizer_training_notebook", framework="none")

## Carga del dataset

# Entrenando tu propio tokenizer desde cero

En este cuaderno, veremos varias formas de entrenar tu propio tokenizer desde cero en un corpus dado, para que luego puedas usarlo para entrenar un modelo de lenguaje desde cero.

¿Por qué necesitarías *entrenar* un tokenizer? Esto se debe a que los modelos Transformer a menudo utilizan algoritmos de tokenización de subpalabras, y necesitan ser entrenados para identificar las partes de las palabras que suelen estar presentes en el corpus que estás utilizando. Te recomendamos que eches un vistazo al [capítulo de tokenización](https://huggingface.co/course/chapter2/4?fw=pt) del curso de Hugging Face para obtener una introducción general sobre los tokenizers, y al [resumen de tokenizers](https://huggingface.co/transformers/tokenizer_summary.html) para conocer las diferencias entre los algoritmos de tokenización de subpalabras.

## Getting a corpus

Necesitaremos textos para entrenar nuestro tokenizer. Utilizaremos la biblioteca [🤗 Datasets](https://github.com/huggingface/datasets) para descargar nuestros datos de texto, lo cual se puede hacer fácilmente con la función `load_dataset`:

In [48]:
from datasets import load_dataset

Para este ejemplo, utilizaremos Wikitext-2 (que contiene 4,5 MB de textos, por lo que el entrenamiento será rápido para nuestro ejemplo), pero puedes utilizar cualquier conjunto de datos que desees (y en cualquier idioma, excepto inglés).

In [49]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

Found cached dataset wikitext (/home/roberto.lopez/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


Podemos echar un vistazo al conjunto de datos, que tiene 36,718 textos:

In [50]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

Para acceder a un elemento, simplemente tenemos que proporcionar su índice:

In [51]:
dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

También podemos acceder a un fragmento directamente, en cuyo caso obtenemos un diccionario con la clave `"text"` y una lista de textos como valor:

In [52]:
dataset[:5]

{'text': ['',
  ' = Valkyria Chronicles III = \n',
  '',
  ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
  " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making th

La API para entrenar nuestro tokenizer requerirá un iterador de lotes de textos, por ejemplo, una lista de listas de textos:

In [53]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

Para evitar cargar todo en memoria (ya que la biblioteca Datasets mantiene los elementos en disco y solo los carga en memoria cuando se solicitan), definimos un iterador de Python. Esto es especialmente útil si tienes un conjunto de datos enorme:

In [54]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

Ahora veamos cómo podemos utilizar este corpus para entrenar un nuevo tokenizer. Hay dos APIs para hacer esto: la primera utiliza un tokenizer existente y entrenará una nueva versión en tu corpus en una línea de código, la segunda es construir realmente tu tokenizer paso a paso, lo que te permite personalizar cada paso.

## Usando un tokenizador ya existente

Si deseas entrenar un tokenizer con los mismos algoritmos y parámetros que uno existente, simplemente puedes utilizar la API `train_new_from_iterator`. Por ejemplo, vamos a entrenar una nueva versión del tokenizador GPT-2 en Wikitext-2 utilizando el mismo algoritmo de tokenización.

Primero necesitamos cargar el tokenizador que queremos usar como modelo:

In [55]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

¡Asegúrate de elegir un tokenizador de versión *fast* (respaldado por la biblioteca 🤗 Tokenizers) o de lo contrario, el resto del cuaderno no se ejecutará correctamente!

In [56]:
tokenizer.is_fast

True

Luego, alimentamos el corpus de entrenamiento (ya sea la lista de listas o el iterador que definimos anteriormente) al método `train_new_from_iterator`. También debemos especificar el tamaño del vocabulario que queremos usar:

In [57]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

¡Y eso es todo! El entrenamiento es muy rápido gracias a la biblioteca 🤗 Tokenizers, respaldada por Rust.

Ahora tienes un nuevo tokenizer listo para preprocesar tus datos y entrenar un modelo de lenguaje. Puedes proporcionarle textos de entrada como de costumbre:

In [58]:
new_tokenizer(dataset[:5]["text"])

{'input_ids': [[], [301, 8639, 9504, 3050, 301, 315], [], [4720, 74, 4825, 889, 8639, 491, 529, 672, 6944, 475, 267, 9504, 374, 2809, 529, 10879, 231, 100, 162, 255, 113, 14391, 4046, 113, 4509, 95, 18351, 4509, 256, 4046, 99, 4046, 22234, 96, 19, 264, 6437, 272, 8639, 281, 261, 3518, 2035, 491, 373, 264, 5162, 3305, 290, 344, 8639, 9504, 3050, 2616, 1822, 264, 364, 259, 14059, 1559, 340, 2393, 1527, 737, 1961, 370, 805, 3604, 288, 7577, 14, 54, 782, 337, 261, 4840, 15585, 272, 19958, 284, 1404, 1696, 284, 1822, 264, 385, 364, 261, 1431, 737, 284, 261, 8639, 906, 272, 2531, 1858, 286, 261, 1112, 9658, 281, 14059, 288, 1626, 340, 645, 6556, 344, 520, 14434, 264, 261, 1485, 3436, 7515, 290, 261, 518, 737, 288, 4750, 261, 302, 22039, 302, 264, 259, 21720, 1743, 3836, 5654, 261, 4259, 281, 4742, 490, 724, 261, 3581, 1351, 283, 1114, 579, 952, 4010, 1985, 2563, 288, 453, 2128, 807, 935, 261, 7655, 3836, 302, 2038, 314, 271, 89, 22414, 302, 272, 315], [324, 737, 1022, 1984, 284, 1525, 264, 7

Puedes guardarlo localmente con el método `save_pretrained`:

In [59]:
new_tokenizer.save_pretrained("my-new-tokenizer")

('my-new-tokenizer/tokenizer_config.json',
 'my-new-tokenizer/special_tokens_map.json',
 'my-new-tokenizer/vocab.json',
 'my-new-tokenizer/merges.txt',
 'my-new-tokenizer/added_tokens.json',
 'my-new-tokenizer/tokenizer.json')

O incluso subirlo al [Hugging Face Hub](https://huggingface.co/models) para usar ese nuevo tokenizer desde cualquier lugar. Asegúrate de tener tu token de autenticación almacenado ejecutando `huggingface-cli login` en una terminal o ejecutando la siguiente celda:

In [60]:
from huggingface_hub import notebook_login

notebook_login()

Estamos casi listos, también es necesario que tengas instalado `git lfs`. Puedes hacerlo directamente desde este cuaderno descomentando las siguientes celdas:

In [61]:
# !apt install git-lfs

In [62]:
new_tokenizer.push_to_hub("my-new-shiny-tokenizer")

CommitInfo(commit_url='https://huggingface.co/robertoLC/my-new-shiny-tokenizer/commit/27c1ed9a5d5b2b6f886c6f8a268507ff9ce9e972', commit_message='Upload tokenizer', commit_description='', oid='27c1ed9a5d5b2b6f886c6f8a268507ff9ce9e972', pr_url=None, pr_revision=None, pr_num=None)

El tokenizer ahora se puede volver a cargar en esta máquina con el siguiente código:

In [63]:
tok = new_tokenizer.from_pretrained("my-new-tokenizer")

O desde cualquier lugar utilizando el ID del repositorio, que es tu espacio de nombres seguido de una barra diagonal y el nombre que le diste en el método `push_to_hub`, por ejemplo:

```python
tok = new_tokenizer.from_pretrained("sgugger/my-new-shiny-tokenizer")
```

Ahora, si deseas crear y entrenar un nuevo tokenizer que no se parezca a nada existente, deberás construirlo desde cero utilizando la biblioteca 🤗 Tokenizers.

## Construyendo el tokenizer desde cero

Para comprender cómo construir tu tokenizer desde cero, debemos adentrarnos un poco más en la biblioteca 🤗 Tokenizers y en la pipeline de tokenización. Esta pipeline consta de varios pasos:

- **Normalización**: Ejecuta todas las transformaciones iniciales sobre la cadena de entrada inicial. Por ejemplo, cuando necesitas convertir un texto en minúsculas, eliminar espacios o incluso aplicar uno de los procesos comunes de normalización Unicode, agregarás un Normalizer.
- **Pre-tokenización**: Se encarga de dividir la cadena de entrada inicial. Esta es la parte que decide dónde y cómo presegmentar la cadena original. El ejemplo más simple sería dividir la cadena por espacios.
- **Modelo**: Se encarga de descubrir y generar todos los subtokens. Esta es la parte que se puede entrenar y que depende realmente de tus datos de entrada.
- **Post-Procesamiento**: Proporciona funciones de construcción avanzadas para ser compatibles con algunos de los modelos más avanzados basados en Transformers. Por ejemplo, para BERT, envolvería la oración tokenizada con los tokens [CLS] y [SEP].

Y para revertir el proceso:

- **Decodificación**: Se encarga de mapear nuevamente una entrada tokenizada a la cadena original. El decodificador se elige generalmente según el `PreTokenizer` que se utilizó anteriormente.

Para entrenar el modelo, la biblioteca 🤗 Tokenizers proporciona una clase `Trainer` que utilizaremos.

Todos estos componentes se pueden combinar para crear pipelines de tokenización funcionales. Para darte algunos ejemplos, mostraremos tres pipelines completas aquí: cómo replicar GPT-2, BERT y T5 (que te darán ejemplos de tokenizadores BPE, WordPiece y Unigram).

### WordPiece model like BERT

Veamos cómo podemos crear un tokenizador WordPiece similar al utilizado para entrenar BERT. El primer paso es crear un `Tokenizer` con un modelo `WordPiece` vacío:

In [64]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

Ignored unknown kwargs option unl_token


Este `tokenizer` aún no está listo para el entrenamiento. Tenemos que agregar algunos pasos de preprocesamiento: la normalización (que es opcional) y el pre-tokenizador, que dividirá las entradas en fragmentos que llamaremos palabras. Los tokens formarán parte de esas palabras (pero no pueden ser más grandes que eso).

En el caso de BERT, la normalización implica convertir todo a minúsculas. Como BERT es un modelo muy popular, tiene su propio normalizador:

In [65]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

Si deseas personalizarlo, puedes utilizar los bloques existentes y componerlos en una secuencia: aquí, por ejemplo, convertimos todo a minúsculas, aplicamos la normalización NFD y eliminamos los acentos:

In [66]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

También existe un `BertPreTokenizer` que podemos utilizar directamente. Realiza una pre-tokenización utilizando espacios en blanco y puntuación:

In [67]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

Al igual que con el normalizador, podemos combinar varios pre-tokenizadores en una secuencia (`Sequence`). Si queremos echar un vistazo rápido a cómo se preprocesan las entradas, podemos llamar al método `pre_tokenize_str`:

In [68]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

Ten en cuenta que el pre-tokenizador no solo divide el texto en palabras, sino que también conserva los desplazamientos, es decir, el inicio y el final de cada una de esas palabras dentro del texto original. Esto permitirá que el tokenizador final pueda asignar cada token a la parte del texto de la que proviene (una característica que utilizamos para tareas de respuesta a preguntas o clasificación de tokens).

Ahora podemos entrenar nuestro tokenizador (aunque el pipeline no esté completamente terminado, necesitaremos un tokenizador entrenado para construir el postprocesador). Para ello, utilizamos un `WordPieceTrainer`. Lo importante es recordar pasar los tokens especiales al entrenador, ya que no se verán en el corpus.

In [69]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

Para entrenar el tokenizador, el método es similar a lo que usamos anteriormente: podemos pasar archivos de texto o un iterador de lotes de textos.

In [70]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

Ahora que el tokenizador está entrenado, podemos definir el post-procesador: necesitamos agregar el token CLS al principio y el token SEP al final (para oraciones individuales) o varios tokens SEP (para pares de oraciones). Usaremos `TemplateProcessing` para hacer esto, lo cual requiere conocer los ID de los tokens CLS y SEP (por eso esperamos al entrenamiento).

Entonces, primero obtengamos los ID de los dos tokens especiales:

In [71]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


Y aquí está cómo podemos construir nuestro post-procesador. Tenemos que indicar en la plantilla cómo organizar los tokens especiales con una oración (`$A`) o dos oraciones (`$A` y `$B`). Los dos puntos seguidos de un número indican el ID del tipo de token a asignar a cada parte.

In [72]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

Podemos verificar si obtenemos los resultados esperados codificando un par de oraciones, por ejemplo:

In [73]:
encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

Podemos ver los tokens para verificar si los tokens especiales se han insertado en los lugares correctos:

In [74]:
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

Y podemos verificar que los ID de tipo de token sean correctos:

In [75]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

La última pieza en este tokenizador es el decodificador, usamos un decodificador `WordPiece` e indicamos el prefijo especial `##`:

In [76]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

Ahora que nuestro tokenizador está terminado, necesitamos envolverlo dentro de un objeto de Transformers para poder usarlo con la biblioteca Transformers. Más específicamente, debemos colocarlo dentro de la clase de tokenizador rápido correspondiente al modelo que queremos utilizar, aquí un `BertTokenizerFast`:

In [77]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

Y como antes, podemos usar este tokenizador como un tokenizador normal de Transformers y usar los métodos `save_pretrained` o `push_to_hub`.

Si el tokenizador que estás construyendo no coincide con ninguna clase en Transformers porque es muy especial, puedes envolverlo en `PreTrainedTokenizerFast`.

### BPE model like GPT-2

Ahora veamos cómo podemos crear un tokenizador BPE similar al utilizado para entrenar GPT-2. El primer paso es crear un objeto `Tokenizer` con un modelo `BPE` vacío:

In [78]:
tokenizer = Tokenizer(models.BPE())

Como antes, tenemos que añadir la normalización opcional (que no se utiliza en el caso de GPT-2) y necesitamos especificar un pre-tokenizador antes del entrenamiento. En el caso de GPT-2, se utiliza un pre-tokenizador a nivel de bytes:

In [79]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

Si queremos echar un vistazo rápido a cómo se preprocesan las entradas, podemos llamar al método `pre_tokenize_str`:

In [80]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

Usamos el mismo valor predeterminado que en GPT-2 para el espacio de prefijo, por lo que puedes ver que cada palabra tiene un `'Ġ'` inicial agregado al principio, excepto la primera.

¡Ahora podemos entrenar nuestro tokenizador! Esta vez usamos un `BpeTrainer`.

In [81]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

Para completar todo el pipeline, tenemos que incluir el post-procesador y el decodificador:

In [ ]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

Y, como antes, terminamos envolviéndolo en un objeto de tokenizer de Transformers:

In [ ]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Unigram model like Albert

Ahora veamos cómo podemos crear un tokenizador de Unigram, similar al utilizado para entrenar T5. El primer paso es crear un objeto `Tokenizer` con un modelo `Unigram` vacío:

In [ ]:
tokenizer = Tokenizer(models.Unigram())

Al igual que antes, debemos agregar la normalización opcional (en este caso, algunas sustituciones y minúsculas) y necesitamos especificar un pre-tokenizador antes del entrenamiento. El pre-tokenizador utilizado es un pre-tokenizador de `Metaspace`: reemplaza todos los espacios por un carácter especial (por defecto, ▁) y luego divide en ese carácter.

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"'), normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

Si queremos ver rápidamente cómo se procesan las entradas, podemos llamar al método `pre_tokenize_str`:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('▁This', (0, 4)), ('▁is', (4, 7)), ('▁an', (7, 10)), ('▁example!', (10, 19))]

Puedes ver que cada palabra tiene un `▁` inicial agregado al principio, como suele hacerse con sentencepiece.

¡Ahora podemos entrenar nuestro tokenizador! Esta vez usamos un `UnigramTrainer`. "Tenemos que establecer explícitamente el token desconocido en este entrenador, de lo contrario, lo olvidará después.

In [ ]:
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=["[CLS]", "[SEP]", "<unk>", "<pad>", "[MASK]"], unk_token="<unk>")
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

Para finalizar todo el proceso, debemos incluir el post-procesador y el decodificador. El post-procesador es muy similar a lo que vimos con BERT, y el decodificador es simplemente `Metaspace`, al igual que el pre-tokenizador.

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)
tokenizer.decoder = decoders.Metaspace()

Y al igual que antes, terminamos envolviendo esto en un objeto de tokenizador de Transformers.

In [ ]:
from transformers import AlbertTokenizerFast

new_tokenizer = AlbertTokenizerFast(tokenizer_object=tokenizer)